In [1]:
%matplotlib inline
import torch
import os,sys
import numpy as np
import torchvision.io as io
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#from torchvision import transforms 
from torch.utils.data import Dataset, random_split

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
if "google.colab" in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True, use_metadata_server=False)

Mounted at /content/drive


In [4]:
def import_file(url, filename):
  import requests
  r = requests.get(url)

  with open(filename, 'w') as f:
        f.write(r.text)

if "google.colab" in sys.modules:
    # Save datagenerators as file to colab working directory
    # If you are using GitHub, make sure you get the "Raw" version of the code
    url_u_net = 'https://raw.githubusercontent.com/karimassi/road-segmentation/u_net/src/models/u_net.py?token=AJVSYDHQIQ5KJXR6G5EH6NS7275YE'
    import_file(url_u_net, 'u_net.py')

    url_u_net_paper = 'https://raw.githubusercontent.com/karimassi/road-segmentation/u_net/src/models/u_net_paper.py?token=AJVSYDDDE2ZOO2E5RKHSDNK7275ZI'
    import_file(url_u_net_paper, 'u_net_paper.py')

    url_training = 'https://raw.githubusercontent.com/karimassi/road-segmentation/u_net/src/training.py?token=AJVSYDBUQJXHKKCMQZTOC3272752S'
    import_file(url_training, 'training.py')

    url_image_mask_dataset = "https://raw.githubusercontent.com/karimassi/road-segmentation/u_net/src/image_mask_dataset.py?token=AJVSYDEYVNI34EBZZW6QMPK72753U"
    import_file(url_image_mask_dataset, 'image_mask_dataset.py')

    url_losses = "https://raw.githubusercontent.com/karimassi/road-segmentation/u_net/src/losses.py?token=AJVSYDGGYYFBALHBYGZQM4S727546"
    import_file(url_losses, 'losses.py')
        
    import u_net
    import u_net_paper 
    import training
    import image_mask_dataset
    import losses
else:
    from src import u_net
    from src import u_net_paper
    from src import training
    from src import image_mask_dataset
    from src import losses

In [9]:
import importlib
importlib.reload(image_mask_dataset)
importlib.reload(u_net)
importlib.reload(u_net_paper)
importlib.reload(training)
importlib.reload(losses)

<module 'losses' from '/content/losses.py'>

In [6]:
# load data
from image_mask_dataset import ImageMaskDataset

root_dir = "/content/drive/Shareddrives/road-segmentation/data/training/"
image_dir = root_dir + "images/"
gt_dir = root_dir + "groundtruth/"

# TODO change implementation of this dataset
dataset = ImageMaskDataset(image_dir, gt_dir, 0)

# apply transformations
rotations = [5, 10, 15, 27, 32, 44, 50, 59, 63, 70, 86, 90, 100, 120, -30, -45, -90]
for angle in rotations:
    dataset += ImageMaskDataset(image_dir, gt_dir, angle)

#dataset += ImageMaskDataset(image_dir, gt_dir, transforms.RandomVerticalFlip(p = 1.0))
#dataset += ImageMaskDataset(image_dir, gt_dir, transforms.RandomHorizontalFlip(p = 1.0))

In [10]:
num_epochs = 50
learning_rate = 1e-3
batch_size = 5

data_len = len(dataset)
train_len = int(data_len * 0.9)
test_len = int(data_len * 0.1)

dataset_train, dataset_test = random_split(dataset, [train_len, test_len])
print(len(dataset_train), len(dataset_test))

dataloader_train = torch.utils.data.DataLoader(
    dataset_train,
    batch_size=batch_size,
    shuffle=True
)

dataloader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=batch_size,
    shuffle=True
)

import u_net_paper
from u_net_paper import UNet_paper

NUM_CHANNELS = 3
NUM_FILTERS = 32

import training

# Train the u-net model with the SGD optimizer
model = UNet_paper(NUM_CHANNELS, NUM_FILTERS).to(device)
criterion = losses.Dice()

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
training.train(model, criterion, dataloader_train, dataloader_test, optimizer, num_epochs)

#from torchsummary import summary
#summary(model, input_size=(NUM_CHANNELS, 400, 400))

1620 180
Starting training
Epoch  1 | Test accuracy : 0.45063 | Test F1 : 0.46159
Epoch  2 | Test accuracy : 0.45044 | Test F1 : 0.49575
Epoch  3 | Test accuracy : 0.45044 | Test F1 : 0.52257
Epoch  4 | Test accuracy : 0.45044 | Test F1 : 0.54131
Epoch  5 | Test accuracy : 0.45044 | Test F1 : 0.55196
Epoch  6 | Test accuracy : 0.45044 | Test F1 : 0.5583
Epoch  7 | Test accuracy : 0.45044 | Test F1 : 0.56315
Epoch  8 | Test accuracy : 0.45044 | Test F1 : 0.56537
Epoch  9 | Test accuracy : 0.45044 | Test F1 : 0.56707
Epoch  10 | Test accuracy : 0.45044 | Test F1 : 0.56801
Epoch  11 | Test accuracy : 0.45044 | Test F1 : 0.5691
Epoch  12 | Test accuracy : 0.45044 | Test F1 : 0.56984
Epoch  13 | Test accuracy : 0.45044 | Test F1 : 0.57021
Epoch  14 | Test accuracy : 0.45044 | Test F1 : 0.57073
Epoch  15 | Test accuracy : 0.45044 | Test F1 : 0.57099
Epoch  16 | Test accuracy : 0.45044 | Test F1 : 0.57138
Epoch  17 | Test accuracy : 0.45044 | Test F1 : 0.57138
Epoch  18 | Test accuracy : 0.45